<a href="https://colab.research.google.com/github/Saifullah785/deep-learning-projects/blob/main/Project_02_Bank_Notes_Detection_Classification_ANN/Project_02_Bank_Notes_Classification_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bank-Notes-Detection-System-Using-ANN**

# **Step 1: Load Tools**

In [15]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import modules for model training and evaluation from scikit-learn
from sklearn.model_selection import train_test_split # For splitting data into training, validation, and testing sets
from sklearn.preprocessing import StandardScaler # For standardizing features
from sklearn.metrics import classification_report, confusion_matrix, roc_curve # For evaluating model performance

# Import modules for building and training the ANN model from TensorFlow/Keras
from tensorflow.keras.models import Sequential # To create a sequential model
from tensorflow.keras.layers import Dense, Dropout # For adding layers to the model
from tensorflow.keras.callbacks import EarlyStopping # For stopping training early
from tensorflow.keras.utils import to_categorical # For one-hot encoding the target variable (if needed, though not used in this specific case for binary classification)

# **Step 2. Load the Dataset**

In [16]:
# Load the dataset from a CSV file named 'train.csv' into a pandas DataFrame
data = pd.read_csv('train.csv')
# Display the first 5 rows of the DataFrame to get a glimpse of the data structure and content
display(data.head())

,VWTI,SWTI,CWTI,EI,Class
0,2.2634,-4.4862,3.6558,-0.612510,0
1,3.2718,1.7837,2.1161,0.613340,0
2,-3.9411,-12.8792,13.0597,-3.312500,1
3,0.5195,-3.2633,3.0895,-0.984900,0
4,2.5698,-4.4076,5.9856,0.078002,0


# **3. Data Preprocessing**

In [17]:
# Separate the features (input variables) and the target variable (output variable)
# X contains the features 'VWTI', 'SWTI', 'CWTI', 'EI'
X = data[['VWTI', 'SWTI', 'CWTI', 'EI']]
# y contains the target variable 'Class'
y = data['Class']

In [18]:
# Split the data into training and testing sets
# 80% of the data is used for training (X_train, y_train)
# 20% of the data is used for testing (X_test, y_test)
# random_state=42 ensures reproducibility of the split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Split the training data further into training and validation sets
# This is done to have a separate set for evaluating the model during training and tuning hyperparameters
# 80% of the original training data is kept as the final training set (X_train, y_train)
# 20% of the original training data is used as the validation set (X_val, y_val)
# random_state=42 ensures reproducibility of this split as well
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [20]:
# Initialize the StandardScaler
scaler = StandardScaler()
# Fit the scaler on the training data to learn the mean and standard deviation of the features
# Then, transform the training data to standardize the features (mean=0, variance=1)
X_train = scaler.fit_transform(X_train)

In [21]:
# Transform the validation data using the scaler fitted on the training data
# It's important to only fit the scaler on the training data to avoid data leakage from the validation/test sets
X_val = scaler.transform(X_val)
# Transform the test data using the same scaler fitted on the training data
X_test = scaler.transform(X_test)

In [22]:
# Convert the target variables from pandas Series to numpy arrays
# This is often required by machine learning models, including those in TensorFlow/Keras
y_train = y_train.values
y_val = y_val.values
y_test = y_test.values

In [23]:
# Print the shapes of the training, validation, and test feature and target sets
# This helps verify that the data splitting and preparation were done correctly
print(f'X_train shape: {X_train.shape}, y_train shape: {y_train.shape}')
print(f'X_val shape: {X_val.shape}, y_val shape: {y_val.shape}')
print(f'X_test shape: {X_test.shape}, y_test shape: {y_test.shape}')

X_train shape: (700, 4), y_train shape: (700,)
X_val shape: (176, 4), y_val shape: (176,)
X_test shape: (220, 4), y_test shape: (220,)
